# Data Analisis Assignment - Group 1
## Introduction
The goal of this project is to analyze hourly electricity generation in Spain, broken down by autonomous community and generation technology. We aim to extract and explore key insights from the data, identify regional and technological trends, and optionally examine correlations between different data sources.
## Task 1: Data Extraction and Saving
The data extraction process is carried out through the Red Eléctrica de España (REE) API. An interactive user interface allows users to input the desired autonomous community and time period. Based on these inputs, the program retrieves the corresponding hourly electricity generation data and exports it to an Excel file named in the format: "Generation_[AutonomousCommunity][StartDate][EndDate]".

__Inputs:__
- Region Code
- Start Date
- End Date

__Output:__
- Excel file with columns: datetime, value by technology, percentage value by technology

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime as dt

# Geo_id per region according to the REE API:
# https://www.ree.es/es/datos/apidatos
# ───────────────────────────────────────────────────────────────────────────
# 1. Dictionary REGIONS  →  readable key  →  API id
# ───────────────────────────────────────────────────────────────────────────
REGIONES = {
    "Andalucía": 4,
    "Aragón": 5,
    "Cantabria": 6,
    "Asturias": 11,
    "Castilla y León": 8,
    "Castilla-La Mancha": 7,
    "Cataluña": 9,
    "Comunidad Valenciana": 15,
    "Extremadura": 16,
    "Galicia": 17,
    "Madrid": 8752,
    "Murcia": 21,
    "Navarra": 14,
    "País Vasco": 10,
    "La Rioja": 20,
    "Islas Baleares": 8743,
    "Islas Canarias": 8742,
    "Ceuta": 8744,
    "Melilla": 8745,
    "Península": 8741,
}

# ───────────────────────────────────────────────────────────────────────────
# 2. Interactive selection (region + dates)
# ───────────────────────────────────────────────────────────────────────────
print("╔══════════════════════════════════════════╗")
print("║   Available Regions (geo_limit=ccaa)     ║")
print("╚══════════════════════════════════════════╝")

for n, reg in enumerate(REGIONES, 1):
    print(f"{n:>2}. {reg}")

# --- Select region ---
while True:
    try:
        idx = int(input("\nNumber of desired region: "))
        region_name = list(REGIONES)[idx - 1]
        geo_id = REGIONES[region_name]
        break
    except (ValueError, IndexError):
        print("⛔ Invalid choice, try again…")

# --- Date input ---
def ask_date(prompt):
    while True:
        try:
            txt = input(prompt)
            return dt.strptime(txt.strip(), "%Y-%m-%d %H:%M")
        except ValueError:
            print("⛔ Invalid format. Example: 2019-01-01 00:00")

start = ask_date("\nStart date (YYYY-MM-DD HH:MM): ")
end   = ask_date("End date   (YYYY-MM-DD HH:MM): ")
if end <= start:
    raise ValueError("End date must be after the start date.")

print(f"\n▶ Region: {region_name}  (geo_id = {geo_id})")
print(f"▶ Period: {start}  →  {end}\n")

# ───────────────────────────────────────────────────────────────────────────
# 3. API call function (daily granularity)
# ───────────────────────────────────────────────────────────────────────────
def get_gen(geo_id, start_date, end_date):
    url = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion"

    headers = {
        "Accept": "application/json",
    }

    params = {
        "start_date": start_date,
        "end_date": end_date,
        "time_trunc": "day",
        "geo_limit":  "ccaa",
        "geo_id": geo_id
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"⛔ Error {response.status_code}")
        print(response.text)
        return pd.DataFrame()  # <- Return empty DataFrame on failure

    data = response.json()

    # Extract generation data
    rows = []
    for technology in data["included"]:
        name = technology["attributes"]["title"]
        for v in technology["attributes"]["values"]:
            rows.append({
                "datetime": v["datetime"],
                "value": v["value"],
                "percentage": v["percentage"],
                "technology": name
            })

    return pd.DataFrame(rows)

# Call API
df = get_gen(geo_id, start, end)


if df.empty:
    print("❗ No data found. Please verify your input.")
else:
    print(df.head())
    # Format dates to string
    start_str = start.strftime("%Y%m%d")
    end_str   = end.strftime("%Y%m%d")

    # Export to Excel
    archivo_excel = f"generacion_{region_name.replace(' ', '_').lower()}_{start_str}_{end_str}.xlsx"
    df.to_excel(archivo_excel, index=False)
    print(f"✅ Data saved to: {archivo_excel}")

## Task 2: Data Preprocessing and Cleaning

__Inputs:__
- dataframe "df" with columns: datetime, value, percentage, technology

__Output:__
- dataframe 'daily_pivot': Containing the daily production values and percentages of each techonlogy
- dataframe 'monthly_pivot': Containing the monthly production values and percentages of each techonlogy

In [ ]:
# Display the top 5 rows of the Dataframe
df.head()

In [ ]:
# Display information about the data including the index dtype and columns, non-null values and memory usage
df.info()

In [ ]:
# Display descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.
df.describe()

In [ ]:
# Check data for missing values
missing_values = df.isnull().sum()

# Filter columns with missing values
missing_cols = missing_values[missing_values > 0]

# Print results
if missing_cols.empty:
    print("No null values found.")
else:
    print("Missing values found in the following columns:")
    print(missing_cols)
    print("\nPossible treatments:")
    print("- Drop rows with missing values: df.dropna(inplace=True)")
    print("- Fill missing values (example: fill with mean): df.fillna(df.mean(), inplace=True)")

In [ ]:
# Convert datetime column
df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")

In [ ]:
# Sort by datetime just in case
df = df.sort_values('datetime')

### Dataframe restructure:
The dataframe 'df' now is structured like in the columns:
- datetime
- value
- percentage
- technology

Note that all the first dates of a month contain two values: one for the actual value and percentage of the technology of that date and one for the cummulated value and percentage of the month.

To fix this the dataframe is restructured into two resulting ones. One with the monthly statistics and one with the daily statistics.

In [ ]:
# Restructure to get the dataframe for the daily data.
# Output structure: datetime	value_[technology]	percentage_[technology]

# Drop the cumulative rows — keep only the first daily entry
daily_df = df.copy()
daily_df['day'] = daily_df['datetime'].dt.date

# For each day and technology, keep only the minimum value (assuming the cumulative has higher value)
daily_df = daily_df.sort_values(['datetime', 'technology', 'value']).drop_duplicates(['day', 'technology'], keep='first')

# Pivot to wide format
daily_pivot = daily_df.pivot(index='datetime', columns='technology', values=['value', 'percentage'])

# Flatten MultiIndex columns
daily_pivot.columns = [f"{col[0]}_{col[1]}" for col in daily_pivot.columns]
daily_pivot = daily_pivot.reset_index()

# Check for the results
daily_pivot.head()

In [ ]:
# Restructure to get the dataframe for the monthly data.
# Output structure: datetime	value_[technology]	percentage_[technology]

# Get cumulative values: assume they are the maximum for each technology on the first day of the month
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.to_period('M')

# Filter to first day of month
monthly_df = df[df['day'] == 1]

# For each month and technology, keep the **max** value (cumulative)
monthly_cum = monthly_df.sort_values(['value'], ascending=False).drop_duplicates(['month', 'technology'], keep='first')

# Pivot to wide format
monthly_pivot = monthly_cum.pivot(index='month', columns='technology', values=['value', 'percentage'])

# Flatten MultiIndex columns
monthly_pivot.columns = [f"{col[0]}_{col[1]}" for col in monthly_pivot.columns]
monthly_pivot = monthly_pivot.reset_index()

# Check for the results
monthly_pivot.head()

In [ ]:
# Check daily data for outliers values
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Assume daily_pivot is already prepared as described earlier
# Exclude 'datetime' column
data = daily_pivot.select_dtypes(include=[np.number])

# -------------------------
# 1. Z-score Method
# -------------------------
z_scores = np.abs(stats.zscore(data, nan_policy='omit'))
z_outliers = (z_scores > 3)

z_outlier_cols = data.columns[(z_outliers.sum(axis=0) > 0)].tolist()

# -------------------------
# 2. IQR Method
# -------------------------
iqr_outlier_cols = []
for col in data.columns:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = data[(data[col] < lower) | (data[col] > upper)]
    if not outliers.empty:
        iqr_outlier_cols.append(col)

# -------------------------
# 3. Boxplot Visualization
# -------------------------
for col in data.columns:
    plt.figure()
    plt.boxplot(data[col].dropna())
    plt.title(f'Boxplot for {col}')
    plt.ylabel(col)
    plt.grid(True)
    plt.show()

# -------------------------
# 4. Summary Output
# -------------------------
if not z_outlier_cols and not iqr_outlier_cols:
    print("✅ No outliers found using Z-score or IQR methods.")
else:
    all_outlier_cols = sorted(set(z_outlier_cols + iqr_outlier_cols))
    print("⚠️ Outliers detected in the following columns:")
    for col in all_outlier_cols:
        print(f"  - {col}")

    # Optional treatment (e.g., replacing with NaN or clipping)
    # Example: Replace extreme Z-score outliers with NaN
    for col in all_outlier_cols:
        daily_pivot.loc[z_outliers[:, data.columns.get_loc(col)], col] = np.nan

    print("\n📌 Treatment applied: Replaced Z-score outliers (>3σ) with NaN.")

In [ ]:
# Check for outliers in monthly data
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Assume monthly_pivot is already prepared
# Remove 'month' column for numeric analysis
data = monthly_pivot.select_dtypes(include=[np.number])

# -------------------------
# 1. Z-score Method
# -------------------------
z_scores = np.abs(stats.zscore(data, nan_policy='omit'))
z_outliers = (z_scores > 3)

z_outlier_cols = data.columns[(z_outliers.sum(axis=0) > 0)].tolist()

# -------------------------
# 2. IQR Method
# -------------------------
iqr_outlier_cols = []
for col in data.columns:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = data[(data[col] < lower) | (data[col] > upper)]
    if not outliers.empty:
        iqr_outlier_cols.append(col)

# -------------------------
# 3. Boxplot Visualization
# -------------------------
for col in data.columns:
    plt.figure()
    plt.boxplot(data[col].dropna())
    plt.title(f'Boxplot for {col}')
    plt.ylabel(col)
    plt.grid(True)
    plt.show()

# -------------------------
# 4. Summary Output
# -------------------------
if not z_outlier_cols and not iqr_outlier_cols:
    print("✅ No outliers found in monthly data using Z-score or IQR methods.")
else:
    all_outlier_cols = sorted(set(z_outlier_cols + iqr_outlier_cols))
    print("⚠️ Outliers detected in the following columns of monthly data:")
    for col in all_outlier_cols:
        print(f"  - {col}")

    # Optional treatment: Replace Z-score outliers with NaN
    for col in all_outlier_cols:
        monthly_pivot.loc[z_outliers[:, data.columns.get_loc(col)], col] = np.nan

    print("\n📌 Treatment applied: Z-score outliers (>3σ) replaced with NaN in monthly data.")

In [ ]:
# Check data for duplicate values in daily_pivot
duplicates = daily_pivot.duplicated()

if not duplicates.any():
    print("✅ No duplicates found in daily data.")
else:
    num_duplicates = duplicates.sum()
    print(f"⚠️ {num_duplicates} duplicate rows found in daily data.")
    daily_pivot = daily_pivot.drop_duplicates()
    print("🧹 Treatment applied: Duplicate rows were removed from daily data.")

In [ ]:
# Check data for duplicate values in monthly_pivot
duplicates = monthly_pivot.duplicated()

if not duplicates.any():
    print("✅ No duplicates found in monthly data.")
else:
    num_duplicates = duplicates.sum()
    print(f"⚠️ {num_duplicates} duplicate rows found in monthly data.")
    monthly_pivot = monthly_pivot.drop_duplicates()
    print("🧹 Treatment applied: Duplicate rows were removed from monthly data.")

In [ ]:
# Check for NaN/null values in daily_pivot
null_counts = daily_pivot.isnull().sum()
total_null_rows = daily_pivot.isnull().any(axis=1).sum()

if total_null_rows == 0:
    print("✅ No NaN/null values found in daily data.")
else:
    print("⚠️ NaN/null values found in the following columns of daily data:")
    print(null_counts[null_counts > 0])
    print(f"🧹 Total rows with NaNs to be removed: {total_null_rows}")

    # Drop rows with any NaNs
    daily_pivot = daily_pivot.dropna()
    print("📌 Treatment applied: Rows with NaN values were dropped from daily data.")

In [ ]:
# Check for NaN/null values in monthly_pivot
null_counts = monthly_pivot.isnull().sum()
total_null_rows = monthly_pivot.isnull().any(axis=1).sum()

if total_null_rows == 0:
    print("✅ No NaN/null values found in monthly data.")
else:
    print("⚠️ NaN/null values found in the following columns of monthly data:")
    print(null_counts[null_counts > 0])
    print(f"🧹 Total rows with NaNs to be removed: {total_null_rows}")

    # Drop rows with any NaNs
    monthly_pivot = monthly_pivot.dropna()
    print("📌 Treatment applied: Rows with NaN values were dropped from monthly data.")

In [ ]:
# Information about daily_pivot
daily_pivot.info()

In [ ]:
# Descriptive statistics about daily_pivot
daily_pivot.describe()

In [ ]:
# Information about monthly_pivot
monthly_pivot.info()

In [ ]:
# Descriptive statistics about monthly_pivot
monthly_pivot.describe()

## Task 3: Data Visualization and Insights Extraction

This code transforms raw time-series (hourly or daily) data into insightful visualizations by aggregating daily generation values by different time periods (day or week) and presenting them by technology type.

__Input:__
Dataframe 'df' with columns:
- datetime
- value
- percentage
- technology

In [ ]:
# Daily Energy Generation Bar Graph by Technology
# Exclusive of last date in range
# This sums hourly values for each day, in case hourly data becomes available in this widget in the future

import matplotlib.pyplot as plt

df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Remove all rows corresponding to the last date in the dataset (the values are abnormally low)
df_truncated = df[df["date"] != df["date"].max()]

# Remove monthly sum values, which are the larger of two values for each tech on the 1st of the month
# Sort so that the larger values come last, and keep only the first
df_truncated = df_truncated.sort_values(["technology", "date", "value"], ascending=[True, True, True])
df_clean = df_truncated.drop_duplicates(subset=["date", "technology"], keep="first")

pivot_df = df_clean.pivot_table(
    index="date",
    columns="technology",
    values="value",
    aggfunc="sum",
    fill_value=0
    )

pivot_df_indiv = pivot_df.drop(columns="Generación total", errors="ignore")

pivot_df_indiv.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    title=f"Daily generation by technology in Spain",
    )

plt.xlabel("Date")
plt.ylabel("Generation (MWh)")
plt.xticks(rotation=45)
plt.legend(title="Technology", loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()

In [ ]:
# Daily Energy Generation Percentage Bar Graph by Technology
# Exclusive of last date in range
# This sums hourly percentages for each day, in case hourly data becomes available in this widget in the future

import matplotlib.pyplot as plt

df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Remove all rows corresponding to the last date in the dataset (the values are abnormally low)
df_truncated = df[df["date"] != df["date"].max()]

# Remove monthly sum percentages, which are the smaller of two values for each tech on the 1st of the month
# Sort so that the smaller percentages come last, and keep only the first
df_truncated = df_truncated.sort_values(["technology", "date", "percentage"], ascending=[True, True, False])
df_clean = df_truncated.drop_duplicates(subset=["date", "technology"], keep="first")

pivot_df = df_clean.pivot_table(
    index="date",
    columns="technology",
    values="percentage",
    aggfunc="sum",
    fill_value=0
    )

pivot_df_indiv = pivot_df.drop(columns="Generación total", errors="ignore")

pivot_df_indiv.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    title=f"Daily generation percentage by technology in Spain",
    )

plt.xlabel("Date")
plt.ylabel("Generation (%)")
plt.xticks(rotation=45)
plt.legend(title="Technology", loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()


In [ ]:
# Weekly Energy Generation Share Bar Graph by Technology
# exclusive of last date in range
# This aggregates percentage values for complete weeks only (starting on Mondays)

import matplotlib.pyplot as plt

df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Remove all rows corresponding to the last date in the dataset (the values are abnormally low)
df_truncated = df[df["date"] != df["date"].max()]

# Remove monthly sum values, which are the larger of two values for each tech on the 1st of the month
# Sort so that the larger values come last, and keep only the first
df_truncated = df_truncated.sort_values(["technology", "date", "value"], ascending=[True, True, True])
df_clean = df_truncated.drop_duplicates(subset=["date", "technology"], keep="first")

# Add weekly period starting on Monday
df_clean["week"] = pd.to_datetime(df_clean["date"]).dt.to_period("W-MON")

# Count number of days per week
week_counts = df_clean.groupby("week")["date"].nunique()
full_weeks = week_counts[week_counts == 7].index

# Keep only data from full weeks
df_clean = df_clean[df_clean["week"].isin(full_weeks)]

# Pivot the cleaned dataframe
pivot_df = df_clean.pivot_table(
    index="week",
    columns="technology",
    values="value",
    aggfunc="sum",
    fill_value=0
)

pivot_df_indiv = pivot_df.drop(columns="Generación total", errors="ignore")

pivot_df_indiv.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    title=f"Weekly generation by technology in Spain",
)

plt.xlabel("Week")
plt.ylabel("Generation (MWh)")
plt.xticks(rotation=45)
plt.legend(title="Technology", loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()

In [ ]:
# Weekly Energy Generation Share Bar Graph by Technology (Percentage)
# exclusive of last date in range
# This aggregates percentage values for complete weeks only (starting on Mondays)

import matplotlib.pyplot as plt

df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Remove all rows corresponding to the last date in the dataset (the values are abnormally low)
df_truncated = df[df["date"] != df["date"].max()]

# Remove monthly sum percentages, which are the smaller of two values for each tech on the 1st of the month
# Sort so that the smaller percentages come last, and keep only the first
df_truncated = df_truncated.sort_values(["technology", "date", "percentage"], ascending=[True, True, False])
df_clean = df_truncated.drop_duplicates(subset=["date", "technology"], keep="first")

# Add weekly period starting on Monday
df_clean["week"] = pd.to_datetime(df_clean["date"]).dt.to_period("W-MON")

# Count number of days per week
week_counts = df_clean.groupby("week")["date"].nunique()
full_weeks = week_counts[week_counts == 7].index

# Keep only data from full weeks
df_clean = df_clean[df_clean["week"].isin(full_weeks)]

# Pivot the cleaned dataframe
pivot_df = df_clean.pivot_table(
    index="week",
    columns="technology",
    values="percentage",
    aggfunc="mean",
    fill_value=0
)

pivot_df_indiv = pivot_df.drop(columns="Generación total", errors="ignore")

pivot_df_indiv.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 6),
    title=f"Weekly generation share by technology in Spain (%)",
)

plt.xlabel("Week")
plt.ylabel("Share of Generation (%)")
plt.xticks(rotation=45)
plt.legend(title="Technology", loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.tight_layout()
plt.show()


In [ ]:
# Daily Energy Generation Line Charts by Technology
# This script lets the user select a technology from the dataset and shows two line charts:
# one for daily total production (in MWh) and one for average daily percentage.
# It groups and sums values by day to visualize trends over time.

import pandas as pd
import matplotlib.pyplot as plt

df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Get unique technologies and create the dynamic menu
tech_list = sorted(df['technology'].unique())

df_trunc = df[df["date"] != df["date"].max()]

df_trunc_val = df_trunc.sort_values(["technology", "date", "value"], ascending=[True, True, True])
df_trunc_val = df_trunc_val.drop_duplicates(subset=["date", "technology"], keep="first")

df_trunc_perc = df_trunc.sort_values(["technology", "date", "percentage"], ascending=[True, True, False])
df_trunc_perc = df_trunc_perc.drop_duplicates(subset=["date", "technology"], keep="first")

# Menu and input loop
selected_tech = None
while selected_tech is None:
    print("\nSelect a technology to generate the chart:\n")
    for idx, tech in enumerate(tech_list, start=1):
        print(f"{idx}. {tech}")

    user_input = input("\nEnter the number of the selected technology: ")
    try:
        choice = int(user_input)
        if 1 <= choice <= len(tech_list):
            selected_tech = tech_list[choice - 1]
        else:
            print("Number out of range. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a number.")

print(f"\nYou selected: {selected_tech}")

df_val_filtered = df_trunc_val[df_trunc_val['technology'] == selected_tech]
df_perc_filtered = df_trunc_perc[df_trunc_perc['technology'] == selected_tech]

grouped_val = df_val_filtered.groupby('date')['value'].sum().reset_index()
grouped_perc = df_perc_filtered.groupby('date')['percentage'].mean().reset_index()

plt.figure(figsize=(14, 6))

plt.subplot(2, 1, 1)
plt.plot(grouped_val['date'], grouped_val['value'], marker='o')
plt.title(f'Daily Production - {selected_tech}')
plt.ylabel('Production (MWh)')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(grouped_perc['date'], grouped_perc['percentage'], marker='o', color='orange')
plt.title(f'Daily Share of Total Production - {selected_tech}')
plt.ylabel('Percentage')
plt.xlabel('Date')
plt.grid(True)

plt.tight_layout()
plt.show()

## Task 4: Comparison of the results of the different sources

__Inputs:__
- Data of Red Eléctrica de España (REE) API
- Wind data from maritime observations in NOOA corresponding to the Spanish coasts (Group 4)